## RAG With Multi Data Source

In [ ]:
## wikipedia Tool
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)
print(wiki.run("What is ChatGPT?"))


Page: ChatGPT
Summary: ChatGPT is a generative artificial intelligence chatbot developed by OpenAI and released on November 30, 2022. It currently uses GPT-5, a generative pre-trained transformer (GPT), to generate text, speech, and images in respons


In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0).split_documents(docs)
vectordb = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vectordb.as_retriever()
retriever

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever,"langsmith_search", "Search the LangChain documentation")

In [ ]:
retriever_tool.name

In [ ]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun


arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())


arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv_tool.name

'arxiv'

In [ ]:
tools = [wiki, arxiv_tool, retriever_tool]

In [ ]:
tools

In [ ]:
from dotenv import load_dotenv

load_dotenv()
import os
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [ ]:
## Prompt Template

from langchain import hub

##get the prompt from hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

In [ ]:
##Agents
from langchain.agents import create_openai_functions_agent
agent=create_openai_functions_agent(llm,tools,prompt)

In [ ]:
agent

In [ ]:
## To run Agent We need Agent Executer
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)
agent_executor

In [ ]:
agent_executor.invoke("Who is ChatGPT?")

In [ ]:
agent_executor.invoke("What is langsmith?")